## Data

I wil analyse the charachteristics of Tórshavn (FO), Aarhus (DK) and Reykjavik (IS).

Therefore I am going to use Foresquare data for these cities. In the analysis I also need population data grouped on age for these cities.

The data from Aarhus can be found here: https://ledelsesinformation.aarhuskommune.dk/aarhus-i-tal/default.aspx?doc=vfs://Global/Befolkning-demografi.xview

The data from the Faroes can be found here: https://statbank.hagstova.fo/pxweb/fo/H2/H2__IB__IB01/fo_aldbygd.px/?rxid=284ebd11-bcb7-4d96-b6fb-77b0cdc37ae4  

The data for Reykjavik can be found here: https://px.hagstofa.is/pxis/pxweb/is/Ibuar/Ibuar__mannfjoldi__2_byggdir__Byggdakjarnarhverfi/MAN03301.px

I will make requests in the city data and group them appropriatly. 